In [105]:
import pandas as pd
rating_data= pd.read_csv('./ml-latest-small/ratings.csv')
raw_movie_data= pd.read_csv('./ml-latest-small//movies.csv')

In [106]:
rating_data.shape

(100836, 4)

In [107]:
raw_movie_data.shape

(9742, 3)

In [108]:
def movie_filter(input_data, year=1950, genres=None):
    data = input_data.copy() 
    
    data['year'] = data['title'].str[-5:-1]
    data['year'] = pd.to_numeric(data['year'], downcast='integer', errors='coerce')
    data = data.dropna(axis=0)    
    
    data['genres'] = data['genres'].str.split('|')
    
    data = data[data['year']>=year]
    if genres==None:
        return data.reset_index(drop=True)
    mask = []
    for genre_ in data['genres']:
        mask.append(True if np.intersect1d(genre_, genres).size > 0 else False) 
            
    data = data[mask]
    
    return data.reset_index(drop=True)

In [109]:
movie_data = movie_filter(raw_movie_data)

In [110]:
movie_data.head()

,movieId,title,genres,year
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1995.0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995.0
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",1995.0
4,5,Father of the Bride Part II (1995),[Comedy],1995.0


In [111]:
def rating_filter(input_data, movie_data, threshold = 100):
    data = input_data.copy()
    
     #movie_data에 없는 평가를 한 경우 뺌
    movie_set = set(movie_data['movieId'].values)
    data = data[data['movieId'].isin(movie_set)]

    #threshold개 이상 평가를 남긴 row만 남김
    x = data['userId'].value_counts()>=threshold
    y = set(x[x].index)
    data = data[data['userId'].isin(y)]
    
    return data.reset_index(drop=True)

In [112]:
rating_data = rating_filter(rating_data,movie_data).drop('timestamp',axis=1)

In [113]:
R_df = rating_data.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)

In [114]:
def input_user_movie(input_pivot_table):
    copy_pivot = input_pivot_table.copy()
    movie = []
    rating = []
    userID = int(input('사용자의 아이디를 입력하세요 : '))
    size = int(input('평점을 작성할 영화의 개수를 입력하세요 : '))
    for i in range(size) :
        m = int(input ('영화의 ID를 입력하세요. : '))
        movie.append(m)
        r = float(input ('영화의 평점을 입력하세요 : '))
        rating.append(r)
    
    new_data = pd.DataFrame({'userId':userID,'movieId':m,'rating':rating})
    
    #입력받은 데이터를 하나의 행으로 만들어준다.
    a = pd.DataFrame(data = [rating], columns =movie , index =[userID])
    
    copy_pivot = copy_pivot.append(a).fillna(0)
    
    
    return copy_pivot , userID, new_data

In [115]:
R_df , userID, new_data = input_user_movie(R_df)

사용자의 아이디를 입력하세요 : 138479
평점을 작성할 영화의 개수를 입력하세요 : 10
영화의 ID를 입력하세요. : 1
영화의 평점을 입력하세요 : 3.5
영화의 ID를 입력하세요. : 2
영화의 평점을 입력하세요 : 5
영화의 ID를 입력하세요. : 3
영화의 평점을 입력하세요 : 4.5
영화의 ID를 입력하세요. : 4
영화의 평점을 입력하세요 : 4
영화의 ID를 입력하세요. : 5
영화의 평점을 입력하세요 : 5
영화의 ID를 입력하세요. : 6
영화의 평점을 입력하세요 : 4
영화의 ID를 입력하세요. : 7
영화의 평점을 입력하세요 : 3
영화의 ID를 입력하세요. : 8
영화의 평점을 입력하세요 : 2
영화의 ID를 입력하세요. : 9
영화의 평점을 입력하세요 : 1
영화의 ID를 입력하세요. : 10
영화의 평점을 입력하세요 : 5


In [116]:
from scipy.sparse.linalg import svds
def make_prediction_df():
    
    #R은 pivot_table을 numpy matrix로 만든 것
    R = R_df.values
    
    #user_rating_mean은 사용자의 평균 평점
    user_ratings_mean = np.mean(R, axis = 1)
    
    #R_demeaned : 사용자-영화 테이블에 대해 사용자 평균 평점을 뺀 것
    R_demeaned = R - user_ratings_mean.reshape(-1, 1)
        
    #U 행렬, sigma 행렬, V전치행렬을 반환.
    #이때 spicy에 있는 svd를 이용한다.
    U, sigma, Vt = svds(R_demeaned, k = 50)
    
    #sigma는 0이 포함되지 않은 값으로만 구성되어 있다.
    #sigma를 0이 포함된 대칭 행렬로 변환한다.
    sigma = np.diag(sigma)
    
    #SVD가 적용되어 분해된 R_demeaned를 원본 행렬로 복구
    #구한 원본 행렬에 사용자 평균 rating을 더해준다.
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1,1)
    
    preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)
    
    return preds_df

In [127]:
import numpy as np
def recommend_movies(predictions_df, userID, movies_df, new_data, num_recommendations=5):
    
    #인덱스로 변환해주어야 해서 -1 
    user_row_number = list(R_df.index).index(userID)
    
    #앞에서 만든 prediction행렬을 사용자 인덱스에 따라 영화 정렬 -> 영화 평점이 높은 순으로 정렬
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    #원본 rating_data에서 user_id에 해당하는 데이터를 뽑아낸다.
    user_data = new_data
    
    #user_data와 원본 영화 데이터를 합친다.
    user_full = (user_data.merge(movies_df, how = 'left', on='movieId').sort_values(['rating'], ascending=False))
    
    #원본 영화 데이터에서 사용자가 본 영화를 제외한 데이터 추출
    #.merge로 sorted_user_predcitions와 user_full데이터를 합친다.
    #.rename으로 컬럼 이름을 바꾸고 정렬한다
    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',left_on='movieId',right_on='index').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False))
                       
    return recommendations['title'][:num_recommendations]

In [121]:
preds_df = make_prediction_df()

In [128]:
prediction= recommend_movies(preds_df,userID, movie_data, new_data, 10)

In [129]:
prediction

321                Lion King, The (1994)
505                       Aladdin (1992)
137    Die Hard: With a Vengeance (1995)
417                 Jurassic Park (1993)
333                         Speed (1994)
122                     Apollo 13 (1995)
397                 Fugitive, The (1993)
324                     Mask, The (1994)
0                       Toy Story (1995)
96                     Braveheart (1995)
Name: title, dtype: object